In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.constants import R_sun, R_jup
import astropy.units as u

import os
import copy

import numpy as np
import pandas as pd

from funcs.model import full_model, aflare, calculate_specific_flare_flux, aflare, calculate_angular_radius
from funcs.flarefit import log_probability, log_probability_2flares, log_probability_2flares2ars
from funcs.helper import calculate_inclination

CWD = "/".join(os.getcwd().split("/")[:-2])

import warnings
warnings.simplefilter("ignore")


import time
tstamp = time.strftime("%d_%m_%Y_%H_%M", time.localtime())

/usr/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
#tstamp = '13_12_2019_10_18'
tstamp

'28_02_2020_05_53'

In [20]:
0.132/2.7*24

1.1733333333333333

In [21]:
lcs = pd.read_csv(f"{CWD}/data/summary/lcs.csv")

props = pd.read_csv(f"{CWD}/data/summary/properties.csv")

lcs = lcs.merge(props, left_on="ID", right_on="id")


lcs.to_csv(f"{CWD}/data/summary/everything.csv", index=False)
target = lcs.iloc[3]
target.Prot_d = target.Prot_d/2
i_mu, i_sigma = calculate_inclination(target)
i_mu, i_sigma = i_mu.to("rad").value, i_sigma.to("rad").value
target

sin(i)=0.9051468538161039


ID                             277539431
QCS                                   12
typ                                  slc
cadence_min                            2
mission                             tess
prefix                               TIC
origin                            custom
h_mission                           TESS
SpT                                   M7
view_start                          1641
view_stop                         1643.5
view_min                             750
view_max                             970
BJDoff                           2457000
tstart                           1641.84
ampl_min                             0.9
ampl_max                             1.7
RA                               163.815
Dec                             -73.9364
J                                  10.63
R                                    NaN
Gaia_G                           14.7381
vsini_kms                             70
e_vsini_kms                          7.5
Prot_d          

In [ ]:
i_mu/np.pi*180

In [ ]:
i_mu

In [ ]:
from altaipony.altai import find_iterative_median
from altaipony.flarelc import FlareLightCurve
#from funcs.custom_detrending import refine_detrended_flux_err
from funcs.multiperiod import remove_sinusoidal, fetch_lightcurve

flck = fetch_lightcurve(target)

t, sflux, model, period = remove_sinusoidal(target, plot=True, save=False,)# cut=[target.view_start-5,target.view_stop+5])
print(t)
F = FlareLightCurve(time=t, detrended_flux=sflux, 
                    detrended_flux_err=np.array(len(t)*[np.nanmean(flck.flux_err)]))
print(F.detrended_flux_err)
flcd = F#refine_detrended_flux_err(F, mask_pos_outliers_sigma=1.5, 
                                    # std_rolling_window_length=15, pad=25)

choice = (t > target.view_start +.25) & (t < target.view_stop-.25) # only one flare in TIC 237 : + 1.56 - 1.
t = t[choice]
flux = sflux[choice]
flux_err = np.nanmean(flcd.detrended_flux_err)
median = np.nanmean(find_iterative_median(F).it_med-10)
phi = (t - t[0])/target.Prot_d * 2 * np.pi

pd.DataFrame({"phi":phi,"flux":flux, "t":t,
              "flux_err":flux_err, "median_":median}).to_csv(f"{CWD}/data/lcs/{tstamp}_{target.ID}.csv",
                                                             index=False)
median

In [ ]:
lc = pd.read_csv(f"{CWD}/data/lcs/{tstamp}_{target.ID}.csv")
phi = lc.phi.values
flux = lc.flux.values
flux_err = lc.flux_err.values
t= lc.t.values
median = lc.median_[0]
median
plt.plot(lc.t, lc.flux)

In [ ]:

phi_a = (target.tstart- t[0]-.0275)/target.Prot_d * 2 * np.pi
theta_a = 15*np.pi/180#target.theta_a_init
a = 2.6
fwhm =  .03/target.Prot_d * 2 * np.pi
qlum = target.tlum * u.erg/u.s
R = target.rad * R_sun
phi0 = -180* np.pi / 180
Fth = calculate_specific_flare_flux("TESS", flaret=1e4)
phi_a_max = phi[-1]
#plt.plot(phi,flux)
radius = calculate_angular_radius(Fth, a, qlum, R,)
theta_a, i_mu, phi_a, radius



plt.figure(figsize=(16,8))
plt.plot(phi,flux)
for i in range(-180,180,10):
    plt.plot(phi, full_model(phi_a, theta_a, a, fwhm, i_mu, phi0=i* np.pi / 180,
               phi=phi, num_pts=50, qlum=qlum,
               Fth=Fth, R=R, median=median), c="r", alpha=.31,)
plt.plot(phi, full_model(phi_a, theta_a, a, fwhm, i_mu, phi0=phi0,
           phi=phi, num_pts=50, qlum=qlum,
           Fth=Fth, R=R, median=median), c="k", alpha=1,)
plt.ylim(2300,2700)

### Save inits

In [ ]:
log_probs = {"log_probability":[6, log_probability],
             "log_probability_2flares":[9, log_probability_2flares],
             "log_probability_2flares2ars":[10, log_probability_2flares2ars],}

log_prob = "log_probability"
nparam = log_probs[log_prob][0]

with open(f"{CWD}/data/summary/inits.csv", "a") as f:
    firstout = ("date,ID,phi_a,theta_a,a,fwhm,Fth,phi0,"\
                "i_mu,i_sigma,radius_deg,R_Rsun,qlum_erg_s,"\
                "median,log_prob,nparam\n")
    out = (f"{tstamp},{target.ID},{phi_a},{theta_a},{a},{fwhm},{Fth.value},"\
           f"{phi0},{i_mu},{i_sigma},{radius},{(R/R_sun).value},{qlum.value},"\
           f"{median},{log_prob},{nparam}\n")
   # f.write(firstout)
    f.write(out)

## Run MCMC

In [ ]:
from funcs.flarefit import (log_probability,
                            log_probability_2flares,
                            log_probability_2flares2ars)

log_probs = {"log_probability":[6, log_probability],
             "log_probability_2flares":[9, log_probability_2flares],
             "log_probability_2flares2ars":[10, log_probability_2flares2ars],}






# Keyword arguments Nsteps = 50000

def run_mcmc(Nsteps=50000, wiggle=1e-3):

    inits = pd.read_csv(f"{CWD}/data/summary/inits.csv")
    target = inits.loc[inits.ID == ID,:]
    ndim = target.nparam
    nwalkers = 32


    inits = np.array([phi_a, theta_a, a, fwhm, i_mu, phi0]) 
    pos = inits * (1. + wiggle * np.random.randn(nwalkers, target.nparam))

    # Set up the backend
    # Don't forget to clear it in case the file already exists
    filename = f"{CWD}/analysis/results/mcmc/{tstamp}_{target.ID}_MCMC.h5"
    backend = emcee.backends.HDFBackend(filename)
    backend.reset(nwalkers, ndim)


    sampler = emcee.EnsembleSampler(nwalkers, target.nparam, log_probability,
                                    args=(phi, flux, flux_err, target.qlum,
                                          target.Fth, target.R_Rsun, target.median,
                                          {"i_mu":target.i_mu,
                                          "i_sigma":target.i_sigma}),
                                    backend=backend)

    sampler.run_mcmc(pos, Nsteps, progress=True, store=True);



if __name__ == "__main__":
# Read ID from keyboard here

# 

In [ ]:
from funcs.flarefit import log_probability
ndim = 6
nwalkers = 32


inits = np.array([phi_a, theta_a, a, fwhm, i_mu, phi0]) 
pos = inits * (1. + 1e-3 * np.random.randn(nwalkers, ndim))

# Set up the backend
# Don't forget to clear it in case the file already exists
filename = f"{CWD}/analysis/results/mcmc/{tstamp}_{target.ID}_MCMC.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, ndim)


sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability,
                                args=(phi, flux, flux_err, qlum, Fth, R, median, {"i_mu":i_mu,
                                      "i_sigma":i_sigma}),
                                backend=backend)

In [ ]:
print(f"MCMC run at {tstamp}\n\nlight curve file: {filename}")

## Resume MCMC after break

In [ ]:
filename = f"{CWD}/analysis/results/mcmc/{tstamp}_{target.ID}_MCMC.h5"

new_backend = emcee.backends.HDFBackend(filename)
print("Initial size: {0}".format(new_backend.iteration))

In [ ]:
tstamp, target.ID

In [ ]:
new_sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, 
                                    args=(phi, flux, flux_err, 
                                          qlum, Fth, R, median, 
                                          {"i_mu":i_mu, "i_sigma":i_sigma, }),
                                    backend=new_backend)
new_sampler.run_mcmc(None, 50000, progress=True, store=True)
print("Final size: {0}".format(new_backend.iteration))